# **EXPORTAÇÃO E ANÁLISE DA DATABASE**

In [20]:
!pip install spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [21]:
#Criar dataframe
treino = pd.read_csv("train.csv")

In [22]:
#Obter informações sobre a database

print("A database \'train.csv\' possui as seguintes colunas: \n \n ", treino.columns , "\n--------------------------------------")
print("Cada coluna da database \'train.csv\' armazena os seguintes tipos de dados: \n \n", treino.dtypes , "\n--------------------------------------")
print("Cada coluna da database \'train.csv\' possui a seguinte soma de valores nulos: \n \n", treino.isnull().sum(), "\n--------------------------------------")

duplicados = treino.duplicated()
linhas_duplicadas = treino[treino.duplicated()]

print("Por fim, há", linhas_duplicadas.shape[0], "linhas duplicadas na database \'SpotifyFeatures.csv\'.")

A database 'train.csv' possui as seguintes colunas: 
 
  Index(['Artist Name', 'Track Name', 'Popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms', 'time_signature', 'Class'],
      dtype='object') 
--------------------------------------
Cada coluna da database 'train.csv' armazena os seguintes tipos de dados: 
 
 Artist Name            object
Track Name             object
Popularity            float64
danceability          float64
energy                float64
key                   float64
loudness              float64
mode                    int64
speechiness           float64
acousticness          float64
instrumentalness      float64
liveness              float64
valence               float64
tempo                 float64
duration_in min/ms    float64
time_signature          int64
Class                   int64
dtype: object 
---------------------

In [23]:
#Verificar a frequência de gêneros na database
frequencies = treino['Class'].value_counts()

print("Cada gênero presente no database aparece no seguinte número de vezes: \n \n", frequencies)

Cada gênero presente no database aparece no seguinte número de vezes: 
 
 10    4949
6     2587
9     2524
8     1854
5     1447
1     1373
2     1272
0      625
7      576
3      402
4      387
Name: Class, dtype: int64


# **TRANSFORMAÇÕES NA DATABASE**

In [24]:
#Removendo as colunas que não influenciam o modelo
treino = treino.drop(['Artist Name', 'Track Name', 'Popularity', 'duration_in min/ms'], axis=1)
print("As colunas restantes são: \n", treino.columns)

As colunas restantes são: 
 Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'Class'],
      dtype='object')


In [25]:
#Verificando valores nulos nas colunas restantes
print(treino.isnull().sum())

danceability           0
energy                 0
key                 2014
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness    4377
liveness               0
valence                0
tempo                  0
time_signature         0
Class                  0
dtype: int64


In [26]:
#Imputando dados em 'key' a partir da abordagem da WEB api Spotify - NULOS = -1
treino['key'] = treino['key'].fillna(-1)

In [27]:
#Limitando a coluna 'instrumentalness' para 4 casas decimais
treino['instrumentalness'] = treino['instrumentalness'].round(4)

#Imputando dados em 'instrumentalness' com a média de cada gênero
for i in range(11):
  media = treino.loc[treino['Class'] == i, 'instrumentalness'].mean()
  media = media.round(4)
  print("Gênero", i, "tem média", media)
  treino.loc[(treino['Class'] == i) & (treino['instrumentalness'].isna()), 'instrumentalness'] = media

#Transformando 'tempo' em inteiro (BPM)
treino['tempo'] = treino['tempo'].round()

Gênero 0 tem média 0.0995
Gênero 1 tem média 0.1992
Gênero 2 tem média 0.0979
Gênero 3 tem média 0.1314
Gênero 4 tem média 0.0127
Gênero 5 tem média 0.0688
Gênero 6 tem média 0.2626
Gênero 7 tem média 0.7404
Gênero 8 tem média 0.2359
Gênero 9 tem média 0.0498
Gênero 10 tem média 0.1128


In [28]:
#Verificando se preenchimento aconteceu com sucesso
print(treino.isnull().sum())

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
Class               0
dtype: int64


# **MODELO DE IA**

In [29]:
#Desenvolvendo modelo de ML
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

#Separando as características e o alvo
X = treino.drop('Class', axis=1)
y = treino['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Criando o modelo de rede neural com os parâmetros fornecidos
clf = MLPClassifier(hidden_layer_sizes=(50, 100, 50), activation='tanh', solver='adam', alpha=0.05, learning_rate='constant', max_iter=200, random_state=42)

#Treinando o modelo
clf.fit(X_train, y_train)

#Fazendo previsões com o conjunto de teste
y_pred = clf.predict(X_test)

#Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia do modelo: {accuracy}')
#print(clf.score(X_test,y_test))

Acurácia do modelo: 0.4473216270282285


In [43]:
#Credenciais da API
client_credentials_manager = SpotifyClientCredentials(client_id='9b1e9ff837434408ae773f93a6facb78', client_secret='67dbfcb123f343b2a6921dbfe243914f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#Obtendo o nome da música do usuário
nome_da_musica = input("Digite o nome da música que você quer pesquisar: ")
results = sp.search(q=nome_da_musica, limit=1)

#Obtendo o nome da música e do artista
musica = results['tracks']['items'][0]['name']
autor = results['tracks']['items'][0]['artists'][0]['name']

#Mostrando o resultado
print("A música que você buscou retornou o resultado: ", musica, " - ", autor)

#Pegando o ID da música encontrada
id_da_musica = results['tracks']['items'][0]['id']

#Obtendo os dados da música
dadosDaMusica = sp.audio_features(id_da_musica)[0]

#Limitando o atributo 'instrumentalness' para apenas 4 casas decimais
dadosDaMusica['instrumentalness'] = round(dadosDaMusica['instrumentalness'], 4)


Digite o nome da música que você quer pesquisar: CHVRCHES - Miracle
A música que você buscou retornou o resultado:  Miracle  -  CHVRCHES


In [44]:
# Mapeando os dados fornecidos para a ordem desejada
caracteristicas_relevantes = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature']
valores = [dadosDaMusica[key] for key in caracteristicas_relevantes]

# Usando a lista de valores para a previsão
predicao = clf.predict([valores])

if predicao == 0:
    categoria_correspondente = "Acoustic/Folk"
elif predicao == 1:
    categoria_correspondente = "Alt Music"
elif predicao == 2:
    categoria_correspondente = "Blues"
elif predicao == 3:
    categoria_correspondente = "Bollywood"
elif predicao == 4:
    categoria_correspondente = "Country"
elif predicao == 5:
    categoria_correspondente = "HipHop"
elif predicao == 6:
    categoria_correspondente = "Indie/Alt"
elif predicao == 7:
    categoria_correspondente = "Instrumental"
elif predicao == 8:
    categoria_correspondente = "Metal"
elif predicao == 9:
    categoria_correspondente = "Pop"
elif predicao == 10:
    categoria_correspondente = "Rock"
else:
    categoria_correspondente = "Categoria Desconhecida"

#Printando resultado
print("A musica", musica, "de", autor, "faz parte do Gênero Musical -", categoria_correspondente, predicao)

A musica Miracle de CHVRCHES faz parte do Gênero Musical - Rock [10]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [45]:
#Obtendo recomendações
recommendations = sp.recommendations(seed_tracks=[id_da_musica],
                                      limit=10,
                                      target_danceability=dadosDaMusica['danceability'],
                                      target_energy=dadosDaMusica['energy'],
                                      target_liveness=dadosDaMusica['liveness'],
                                      target_valence=dadosDaMusica['valence'])


# Imprimir os nomes das músicas recomendadas e seus artistas
print("Músicas qcom a mesma \"vibe\": \n ")
for track in recommendations['tracks'][1:]:
    print(f"\t Música: {track['name']} — {track['artists'][0]['name']} \n")

Músicas qcom a mesma "vibe": 
 
	 Música: Miracle — CHVRCHES 

	 Música: Wake Me Up — Fickle Friends 

	 Música: ...Ready For It? — Taylor Swift 

	 Música: i'm so tired... — Lauv 

	 Música: Superposition — Young the Giant 

	 Música: Sunday Morning (feat. Josie Dunne) — Matoma 

	 Música: Your Love — HAERTS 

	 Música: Need Your Love (with Noah Kahan) — Gryffin 

	 Música: All of You — Betty Who 

